## Purpose: To preprocess/clean all csv files converted from xmi for RE Sites of Metastases model training

RE model training (required entity pairs):
- cancer imaging findings - body part
- cancer imaging findings - anatomical descriptor
- anatomical descriptor - body part
- direction - body part
- direction - anatomical descriptor

refer: https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/10.Clinical_Relation_Extraction.ipynb#scrollTo=6doZTPX_xnEm

### Note: Before running this notebook, please configure the following paths

In [ ]:
# configure folder path
file_path = r'dataset\02csv'
input_train_folder = 'csv_train'
input_test_folder = 'csv_test'
print(file_path+'\\'+input_train_folder)
print(file_path+'\\'+input_test_folder)

# dataset file name
dataset_name="train4522

## Import Libraries

In [ ]:
import json, os, re, datetime, time
import pandas as pd
import numpy as np
import glob
import os

## Import and merge all csv files

In [ ]:
# function to check for valid entity pair
def check_pair(df, p):
    temp = df[df['pairs'] == p]['check'].values
    #print("***",temp)
    if len(temp) > 0 :
        #print(df[df['pairs'] == p]['relation'][0])
        return temp[0]
    else:
        return "unknown"

In [ ]:
# read all train/test csv files into dataframe
all_train_files = glob.glob(os.path.join(file_path, input_train_folder, "*.csv"))
all_test_files = glob.glob(os.path.join(file_path, input_test_folder, "*.csv"))

df_train_csv = pd.concat((pd.read_csv(f) for f in all_train_files))
df_train_csv = df_train_csv.drop('Unnamed: 0',axis=1)
df_train_csv.reset_index(drop=True, inplace=True)

df_test_csv = pd.concat((pd.read_csv(f) for f in all_test_files))
df_test_csv = df_test_csv.drop('Unnamed: 0',axis=1)
df_test_csv.reset_index(drop=True, inplace=True)

# remove \r\n in sentence column
df_train_csv['sentence'].replace('\n', '', regex=True, inplace=True)
df_train_csv['sentence'].replace('\r', '', regex=True, inplace=True)
df_test_csv['sentence'].replace('\n', '', regex=True, inplace=True)
df_test_csv['sentence'].replace('\r', '', regex=True, inplace=True)

# add column for entity pairs
df_train_csv['pairs'] = (df_train_csv['entity1']+'-'+df_train_csv['entity2']).str.lower()
df_test_csv['pairs'] = (df_test_csv['entity1']+'-'+df_test_csv['entity2']).str.lower()

In [ ]:
# merge df_train_csv and df_test_csv to df_csv for subsequent processing
# add column dataset to indicate train/test

df_train_csv["dataset"]="train"
df_test_csv["dataset"]="test"
df_csv = pd.concat([df_train_csv,df_test_csv])

In [ ]:
df_csv.groupby("dataset").count()

In [ ]:
df_csv.head()

## check for correct entity pairs

In [ ]:
# file: relation_group.csv contains all annotated pairs extracted from train csv
# based on data checking, there are pairs annotated wrongly or in the wrong direction. These will be excluded from training.
# for training of sites of mets, we are only looking at specific pairs, this is indicated in the column sites_of_mets_group
# for better accuracy, we may also combine multiple pairs into same relation, 
# eg direction-body part and direction-anatomcal descriptor are combined as direction-of
# the relation name will be used in RE visualisation, so cannot be too long (max 30 chars)

df_relation = pd.read_csv(os.path.join(file_path,"relation_group.csv"))
df_relation.head()


In [ ]:
df_csv['check'] = df_csv.apply(lambda row: check_pair(df_relation,row['pairs']), axis=1)

df_csv.head(2)

In [ ]:
# save to file for checking
df_csv.to_csv(os.path.join(file_path,"radio_re_"+dataset_name+"_allrelations_check.csv"), index=False)

## Data Cleaning (drop wrong/reverse/unknown pairs)

In [ ]:
# if there are unknown entity pairs, check with annotation team and update the relation_group.csv accordingly
# this can happen if they annotate new reports with new entity pairs which is not found in current file
# currently, there is no unknown cases

In [ ]:
# 27-Jul-2022 check for pairs with unknown/wrong/reverse relation tag, drop them
df_csv[df_csv['check'].isin(['unknown','wrong','reverse'])].index

In [ ]:
df_csv_clean = df_csv.copy()

print("count before drop:", df_csv_clean['check'].count())
df_csv_clean.drop(df_csv_clean[df_csv_clean['check'].isin(['unknown','wrong','reverse'])].index,inplace=True)
print("count after drop:", df_csv_clean['check'].count())

In [ ]:
df_csv_clean.to_csv(os.path.join(file_path,"radio_re_"+dataset_name+"_allrelations_clean.csv"), index=False)

## Extract required pairs for RE Sites of Mets Training

In [ ]:
df_mets = df_relation[pd.notnull(df_relation['sites_of_mets_group'])][['pairs','sites_of_mets_group']]
df_csv_clean = pd.merge(df_csv_clean, df_mets, on='pairs', how='inner')

In [ ]:
df_csv_clean.count()

In [ ]:
# use pair-pair as relation name, not re-grouping
df_csv_clean['relation'] = df_csv_clean['sites_of_mets_group']
df_csv_clean.head(1)

In [ ]:
df_csv_clean.isnull().sum()

In [ ]:
# drop rows with null relation, entity1, entity2
df_csv_clean = df_csv_clean.dropna(axis=0, subset=['relation'])
df_csv_clean = df_csv_clean.dropna(axis=0, subset=['entity1'])
df_csv_clean = df_csv_clean.dropna(axis=0, subset=['entity2'])
df_csv_clean.count()

In [ ]:
df_csv_clean = df_csv_clean.reset_index()

# extract required columns
columns = ["relation","pairs","entity1","chunk1","entity2","chunk2","entity1_begin","entity1_end","entity2_begin","entity2_end","doc_text","doc_title","dataset"]
df_csv_clean = df_csv_clean[columns]

In [ ]:
df_csv_clean.head(2)

In [ ]:
df_csv_clean.groupby('dataset').count()['relation']

In [ ]:
df_csv_clean.groupby(['dataset','relation']).count()['pairs']

In [ ]:
# save to csv for training pipeline
df_csv_clean.to_csv(os.path.join(file_path,"radio_re_"+dataset_name+"_sitesofmets_relations_clean.csv"), index=False)